In [1]:
!pip install sacrebleu==2.5.1
!pip install jiwer==4.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 93.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [sacrebleu]/4 [colorama]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [jiwer]32m1/2 [jiwer]


In [36]:
import jiwer
import sacrebleu
from datetime import datetime
import json
import os
from tqdm import tqdm

In [45]:
all_metrics = []
outdir = 'outputs'
scoredir = 'scores'
for model in os.listdir(outdir):
    for lang in os.listdir(os.path.join(outdir,model)):
        for filename in os.listdir(os.path.join(outdir,model,lang)):
            jsonfile = os.path.join(outdir,model,lang,filename)
            dt = datetime.strptime(filename.lstrip('results_test_').rstrip('.jsonl'), "%Y%m%d_%H%M%S")
            formatted = dt.strftime("%Y-%m-%d %H:%M:%S")
            data = []
            with open(jsonfile, "r", encoding="utf-8") as f:
                for line in f:
                    data.append(json.loads(line))
            all_references = [i['metadata']['label_text'] for i in data]
            all_predictions = [i['response'] for i in data]
            wer = jiwer.wer(all_references, all_predictions)
            bleu = sacrebleu.corpus_bleu(all_predictions, [[ref] for ref in all_references])
            chrfpp = sacrebleu.corpus_chrf(all_predictions, [[ref] for ref in all_references], word_order=2)

            metrics = {
                "experiment_datetime":formatted,
                "model":model,
                "language":lang.title(),
                "wer": wer * 100,
                "bleu": bleu.score,
                "chrF++": chrfpp.score}
            
            all_metrics.append(metrics)
os.makedirs(scoredir,exist_ok=True)
json.dump(all_metrics,open(f'{scoredir}/image_to_text_transliteration.json','w'),indent=4,ensure_ascii=False)